In [1]:
!pip install -U langchain-text-splitters langchain-community

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [11]:
loader = PyPDFLoader("../content/sample_data/assurance.pdf") # Remplace par le vrai nom de ton fichier
pdf = loader.load()

print(f"Document chargé : {len(pdf)} pages trouvées.")

Document chargé : 8 pages trouvées.


In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,      # Taille cible de chaque morceau
    chunk_overlap = 150,    # Chevauchement
    separators = ["\n\n", "\n", " ", ""] # L'ordre de priorité pour couper
)

chunks = text_splitter.split_documents(pdf)

In [13]:
chunks[0].page_content

"M. Tuan Vu Tran \n2 route de Narbonne \nN° App: 167\nRésidence: Résidence Bordegrande \nCplt adresse:\n31320 AUZEVILLE-TOLOSANE\nVotre numéro d’adhésion\xa0: ADHE-20221128-2506\nNous contacter\xa0: etudiant@adh-assurances.fr\nLa Compagnie L'EQUITE, Société Anonyme au capital de 26\xa0469 320 euros - Entreprise régie par le Code des assurances B 572 084 \n697 RCS Paris - Siège Social : 2, rue Pillet-Will 75009 Paris. Société appartenant au Groupe Generali immatriculé sur le registre italien \ndes groupes d'assurances sous le numéro 026, certifie que l'assuré :\nM. Tuan Vu Tran né(e) le 02/04/1998 a souscrit une adhésion dont les références sont les suivantes\xa0: \nNuméro d’adhésion\xa0: ADHE-20221128-2506\nContrat groupe\xa0: AC482864.\nColocataire(s) du logement\xa0: \nAUCUN COLOCATAIRE\nLa présente attestation est valable pour la période comprise entre le 28/11/2022 (17:51) et le 31/08/2023. \nCe contrat est SANS TACITE reconduction, et à renouveler chaque année avant le 1er septemb

In [14]:
print(f"Nombre total de chunks pour {len(pdf)} pages : {len(chunks)}")

lengths = [len(c.page_content) for c in chunks]
import statistics
print(f"Taille moyenne : {statistics.mean(lengths)} caractères")

# Voir à quelle page appartient le chunk n°10
print(f"Le chunk 10 vient de la page : {chunks[10].metadata['page']}")

Nombre total de chunks pour 8 pages : 31
Taille moyenne : 803.7096774193549 caractères
Le chunk 10 vient de la page : 2


In [51]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

hf_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# Création de la base de données physique
vectordb = Chroma.from_documents(
    documents=chunks,
    embedding=hf_embeddings,
    persist_directory="./chroma_db"
)

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [57]:
question = "Quelles sont les garanties de mon contrat ?"
question_embedding = hf_embeddings.embed_documents([question])

# Demande les 'k' meilleurs résultats
# Chroma fait l'embedding de la question et le calcul de similarité
docs = vectordb.similarity_search(question, k=1)

print(f"Question : {question}")
print("---")
print(f"Réponse: {docs[0].page_content}")

Question : Quelles sont les garanties de mon contrat ?
---
Réponse: Ce contrat est SANS TACITE reconduction, et à renouveler chaque année avant le 1er septembre.
Il garantit l'assuré en tant que locataire (ou occupant à titre gratuit), de Appartement situé(e) à l’adresse suivante :
2 route de Narbonne – N° App: 167  - Résidence: Résidence Bordegrande – Cplt adresse :  – Code postal et Ville : 31320 AUZEVILLE-
TOLOSANE 
Il couvre les risques suivants : 
- INCENDIE (sans franchise)
- DEGATS DES EAUX (sans franchise)
- PROTECTION JURIDIQUE
- INDEMNITE REDOUBLEMENT (garantie acquise uniquement pour les étudiants)
- INDIVIDUELLE ACCIDENT (garantie acquise uniquement pour les jeunes diplômés)
Pour toutes les questions relatives à la gestion de votre contrat, à vos déclarations de sinistre ou à vos besoins d’assistance, vous pouvez
contacter le n° 03 20 33 09 33.
La compagnie atteste avoir pris connaissance du fait que les personnes assurées au contrat seront amenées au sein du logement assur